<a href="https://colab.research.google.com/github/christiejibaraki/CUREBench/blob/main/notebooks/2-eval_gpt_oss_20b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run CUREBench eval
- run prediction
- run eval

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
import os
import sys
import os.path

In [3]:
os.environ['BNB_CUDA_VERSION'] = '125'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Setup
- Clone forked CUREBench repo onto local `content` folder (this is not persistent)
- Create virtual environment and install packages
  - Package installation takes about **5 min** ⏰
- Add virtual environment's site-packages to notebook's system path

In [4]:
!git clone https://github.com/christiejibaraki/CUREBench.git

Cloning into 'CUREBench'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 178 (delta 54), reused 34 (delta 22), pack-reused 89 (from 2)
Receiving objects: 100% (178/178), 2.99 MiB | 5.86 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [4]:
%cd CUREBench

/content/CUREBench


In [5]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 385 bytes | 192.00 KiB/s, done.
From https://github.com/christiejibaraki/CUREBench
   5809546..844e453  main       -> origin/main
Updating 5809546..844e453
Fast-forward
 core/eval_framework.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [7]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 35.8 MB/s eta 0:00:00


In [8]:
!virtualenv env

created virtual environment CPython3.12.12.final.0-64 in 249ms
  creator CPython3Posix(dest=/content/CUREBench/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [7]:
!./env/bin/pip list

Package                       Version
----------------------------- -----------
accelerate                    1.10.1
acres                         0.5.0
admet_ai                      1.2.0
aiohappyeyeballs              2.6.1
aiohttp                       3.13.0
aiosignal                     1.4.0
alabaster                     1.0.0
annotated-types               0.7.0
anyio                         4.11.0
attrs                         25.4.0
Authlib                       1.6.5
azure-ai-documentintelligence 1.0.2
azure-core                    1.36.0
azure-identity                1.25.1
babel                         2.17.0
beautifulsoup4                4.14.2
bitsandbytes                  0.48.1
blinker                       1.9.0
cachetools                    6.2.1
certifi                       2025.10.5
cffi                          2.0.0
charset-normalizer            3.4.4
chemfunc                      1.0.12
chemprop                      1.6.1
ci-info                       0.3.0
click 

In [10]:
!./env/bin/pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 115.4 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 29.7 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 166.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 180.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 90.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 29.9 MB/s  0:00:12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 148.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

In [5]:
env_path = './env/lib/python3.12/site-packages'
sys.path.append(env_path)

### read config

In [6]:
import json

In [7]:
config_path= "metadata_config_val.json"
config = json.load(open(config_path, 'r')) if config_path else {}
if 'dataset' in config:
    dataset_config = config['dataset']
    dataset_name = dataset_config.get('dataset_name', 'treatment')
print(f"\nconfig file: {config_path}\ncontents:\n{dataset_config}")
dataset_path = dataset_config.get("dataset_path")


config file: metadata_config_val.json
contents:
{'dataset_name': 'cure_bench_phase1_val', 'dataset_path': 'resources/curebench_valset_pharse1.jsonl', 'description': 'CureBench 2025 val questions'}


In [8]:
config

{'metadata': {'model_name': 'gpt-oss-20b',
  'model_type': 'Open weight model',
  'track': 'internal_reasoning',
  'base_model_type': 'Open weight model',
  'base_model_name': 'gpt-oss-20b',
  'dataset': 'cure_bench_phase1_val',
  'additional_info': 'Submission using configuration file'},
 'dataset': {'dataset_name': 'cure_bench_phase1_val',
  'dataset_path': 'resources/curebench_valset_pharse1.jsonl',
  'description': 'CureBench 2025 val questions'},
 'output_dir': 'competition_test_results'}

### load model with competition kit

In [9]:
from core.eval_framework import CompetitionKit, load_and_merge_config, create_metadata_parser

In [10]:
model_name = "gpt-oss-20b" # config['metadata']['model_name']
print(f"model name: {model_name}")
model_class = 'auto'

model name: gpt-oss-20b


In [11]:
kit = CompetitionKit(config_path=config_path)

In [12]:
print(f"Loading model: {model_name}")
kit.load_model(model_name, model_class)

Loading model: gpt-oss-20b


`torch_dtype` is deprecated! Use `dtype` instead!
This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=



Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

In [13]:
kit.model.system_identity

'\nYou are an expert medical assistant specializing in answering questions.\n\n**Your communication MUST strictly adhere to the Harmony channels:**\n1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.\n2.  **final:** Use this channel for the final output intended for the user.\n\n**Output Rule is Conditional:**\n* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.\n    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)\n* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.\n\n**Instruction:** Generate a complete response. The final output must use the \'final\' channel and adhere to the conditional format rule.'

In [14]:
kit.list_datasets()

Available Datasets:
--------------------------------------------------
  cure_bench_phase1_val: CureBench 2025 val questions


### run model with evaluate
- takes about 23 seconds to do one example on L4 gpu

In [18]:
subset_size = 3
print(f"Running evaluation on dataset: {dataset_name} (subset-size={subset_size})")
results = kit.evaluate(dataset_name, subset_size=subset_size)

Running evaluation on dataset: cure_bench_phase1_val (subset-size=3)
dataset_path: resources/curebench_valset_pharse1.jsonl
CureBenchDataset initialized with 459 examples


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

stopping criteria: [<core.eval_framework.CustomStopStringCriteria object at 0x7bf13668a2d0>]
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look

ERROR:root:[Harmony parse error] Unexpected token 200002 while expecting start token 200006
Evaluating:  33%|███▎      | 1/3 [01:43<03:27, 103.78s/it]

look back window: 10
expected_answer: A
stopping criteria: [<core.eval_framework.CustomStopStringCriteria object at 0x7bf135f92ed0>]
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
parsed: [Message(author=Author(role=<Role.ASSISTANT: 'assistant'>, name=None), content=[TextContent(text='Need open-ended answer.')], channel='analysis', recipient=None, content_type=None)]
stopping criteria: [<core.eval_framework.CustomStopStringCriteria object at 0x7bf1360ead80>]
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window:

...

look back window: 10
expected_answer: B
stopping criteria: [<core.eval_framework.CustomStopStringCriteria object at 0x7bf13c42d400>]
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look back window: 10
look b

ERROR:root:[Harmony parse error] Unexpected token 200002 while expecting start token 200006


look back window: 10
look back window: 10
stopping criteria: [<core.eval_framework.CustomStopStringCriteria object at 0x7bf135fae090>]
look back window: 10


ERROR:root:[Harmony parse error] Unexpected EOS while waiting for message header to complete
Evaluating: 100%|██████████| 3/3 [05:25<00:00, 108.59s/it]

expected_answer: B


In [19]:
results

EvaluationResult(dataset_name='cure_bench_phase1_val', model_name='gpt-oss-20b', accuracy=0.3333333333333333, correct_predictions=1, total_examples=3, predictions=[{'choice': 'A', 'open_ended_answer': 'analysisMCQ. Acne treatment brands: salicylic acid is a salicylic acid product used for acne, but brand name? Usually Clearasil contains salicylic acid. The question: "Which drug brand name is associated with the treatment of acne?" They give options: A: Salicylic Acid, B: Minoxidil, C: Ketoconazole, D: Fluocinonide. Among these, Salicylic Acid is used for acne. So answer A.assistantfinal{"answer":"A"}analysisThe user asks: "Which drug brand name is associated with the treatment of acne?" Options are:\n\nA: Salicylic Acid\n\nB: Minoxidil\n\nC: Ketoconazole\n\nD: Fluocinonide\n\nThe correct answer: "Salicylic Acid". It\'s known for acne treatment.assistantfinal{"answer":"A"} The task: The user wrote a multi-choice question: "Which drug brand name is associated with the treatment of acne?"

### run model manually

In [18]:
import pandas as pd
from typing import Dict, List
from torch.utils.data import DataLoader
from dataset_utils import build_dataset
from core.eval_framework import GPTOSS20BModel
from openai_harmony import load_harmony_encoding

In [19]:
def load_dataset_by_config(config_path):
  # load config file to get dataset info
  config = json.load(open(config_path, 'r')) if config_path else {}
  if 'dataset' in config:
      dataset_config = config['dataset']
      dataset_name = dataset_config.get('dataset_name', 'treatment')
  print(f"\nconfig file: {config_path}\ncontents:\n{dataset_config}")
  dataset_path = dataset_config.get("dataset_path")

  # build dataset
  dataset = build_dataset(
        dataset_config.get("dataset_path"),
    )
  dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
  dataset_list = []

  for batch in dataloader:
      question_type = batch[0][0]

      if question_type == "multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
      elif question_type == "open_ended_multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
              "meta_question": batch[4][0],
          })
      elif question_type == "open_ended":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
  return dataset_list

In [13]:
val_data_config_path= "metadata_config_val.json"
val_data_list = load_dataset_by_config(val_data_config_path)


config file: metadata_config_val.json
contents:
{'dataset_name': 'cure_bench_phase1_val', 'dataset_path': 'resources/curebench_valset_pharse1.jsonl', 'description': 'CureBench 2025 val questions'}
dataset_path: resources/curebench_valset_pharse1.jsonl
CureBenchDataset initialized with 459 examples


In [14]:
dataset = val_data_list[:10]

In [15]:
dataset

[{'question_type': 'multi_choice',
  'id': 'U9PHZ83RKYV8',
  'question': 'Which drug brand name is associated with the treatment of acne?\nA: Salicylic Acid\nB: Minoxidil\nC: Ketoconazole\nD: Fluocinonide',
  'answer': 'A'},
 {'question_type': 'open_ended_multi_choice',
  'id': 'vIGwm8qguXYi',
  'question': 'What should patients do if they experience severe allergic reactions during or after receiving fosaprepitant for injection?',
  'answer': 'B',
  'meta_question': "The following is a multiple choice question about medicine and the agent's open-ended answer to the question. Convert the agent's answer to the final answer format using the corresponding option label, e.g., 'A', 'B', 'C', 'D', 'E' or 'None'. \n\nQuestion: What should patients do if they experience severe allergic reactions during or after receiving fosaprepitant for injection?\nA: Wait for the symptoms to resolve on their own.\nB: Inform their healthcare provider immediately and seek emergency medical care.\nC: Stop chem

In [45]:
example = dataset[3]
print(example)

{'question_type': 'open_ended_multi_choice', 'id': 'WfWiWK0yULaX', 'question': 'Which of the following conditions is a contraindication for the use of Gadavist?', 'answer': 'B', 'meta_question': "The following is a multiple choice question about medicine and the agent's open-ended answer to the question. Convert the agent's answer to the final answer format using the corresponding option label, e.g., 'A', 'B', 'C', 'D', 'E' or 'None'. \n\nQuestion: Which of the following conditions is a contraindication for the use of Gadavist?\nA: Mild hypersensitivity reactions to Gadavist\nB: History of severe hypersensitivity reactions to Gadavist\nC: Renal impairment\nD: Liver dysfunction\n\n"}


In [46]:
question = example["question"]
question_type = example["question_type"]

In [47]:
# Format prompt
if question_type == "multi_choice":
    prompt = f"The following is a multiple choice question about medicine. Answer with a valid json containing the letter corresponding to the correct answer.\n\nQuestion: {question}\n\nAnswer:"
elif question_type == "open_ended_multi_choice" or question_type == "open_ended":
    prompt = f"The following is an open-ended question about medicine. Provide a comprehensive answer.\n\nQuestion: {question}\n\nAnswer:"

In [48]:
print(f"prompt: {prompt}")

prompt: The following is an open-ended question about medicine. Provide a comprehensive answer.

Question: Which of the following conditions is a contraindication for the use of Gadavist?

Answer:


In [20]:
system_identity = """
You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule."""

In [ ]:
# model = kit.model
model = GPTOSS20BModel("openai/gpt-oss-20b", reasoning_lvl="low", quantization="auto",
                       system_identity=system_identity)
model.load()

In [31]:
# Pass this list to the stop_tokens argument
stop_sequences = [
    # 1. Stops the loop of internal reasoning tags
    "assistantfinal",

    # 2. Stops the common repetitive noise (often necessary)
    "analysisdone."
]

In [49]:
response, reasoning_trace = model.inference(prompt,
    stop_strings=stop_sequences )

In [50]:
response

'**Gadavist (gadobutrol)** is a gadolinium‑based contrast agent commonly used for magnetic resonance imaging (MRI) of the brain and spine. The safety of Gadavist, like all gadolinium‑based contrast agents, is contingent upon adequate renal clearance. **Severe or advanced renal impairment is the principal contraindication** because it predisposes patients to the rare but serious condition known as **Nephrogenic Systemic Fibrosis (NSF)** and increases the risk of gadolinium deposition.\n\nKey points:\n\n| Condition | Why it’s a contraindication for Gadavist | Typical guidelines |\n|-----------|------------------------------------------|--------------------|\n| **Severe chronic kidney disease (CKD Stage 4–5)** – eGFR < 30\u202fmL/min/1.73\u202fm² or requiring dialysis | Reduced elimination of gadobutrol → higher risk of NSF and gadolinium‑related toxicity | Avoid unless essential; if needed, use the lowest effective dose and consider alternative imaging modalities |\n| **Acute kidney inju

In [51]:
reasoning_trace

[{'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text',
    'text': 'Answer is open-ended? They phrased "Which of the following conditions is a contraindication for the use of Gadavist?" But no options given. Likely expecting mention kidney disease (renal impairment). So provide narrative.'}],
  'channel': 'analysis'},
 {'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text',
    'text': '**Gadavist (gadobutrol)** is a gadolinium‑based contrast agent commonly used for magnetic resonance imaging (MRI) of the brain and spine. The safety of Gadavist, like all gadolinium‑based contrast agents, is contingent upon adequate renal clearance. **Severe or advanced renal impairment is the principal contraindication** because it predisposes patients to the rare but serious condition known as **Nephrogenic Systemic Fibrosis (NSF)** and increases the risk of gadolinium deposition.\n\nKey points:\n\n| Condition | Why it’s a contraindicatio

In [28]:
print(model.system_identity)


You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule.


### write submission to file

In [ ]:
# Generate submission with metadata from config/args
print("Generating submission with metadata...")
submission_path = kit.save_submission_with_metadata(
    results=[results],
    filename="submission.csv",
    config_path=config_path,
    args=""
)

print(f"\n✅ Evaluation completed successfully!")
print(f"📊 Accuracy: {results.accuracy:.2%} ({results.correct_predictions}/{results.total_examples})")
print(f"📄 Submission saved to: {submission_path}")

# # Show metadata summary if verbose
# final_metadata = kit.get_metadata(getattr(args, 'config', None), args)
# print("\n📋 Final metadata:")
# for key, value in final_metadata.items():
#     print(f"  {key}: {value}")


### stopping criteria

In [18]:
from transformers import StoppingCriteria, StoppingCriteriaList

In [19]:
stopping_criteria = StoppingCriteriaList()

In [22]:
StoppingCriteria

transformers.generation.stopping_criteria.StoppingCriteria

In [23]:
stopping_criteria

[]

In [24]:
kit.model.enc